In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset = pd.read_csv("CKD.csv")


In [3]:
dataset=pd.get_dummies(dataset,drop_first=True)
dataset



,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.000000,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,0,0,0,0,0,0,1,1,0,1
1,3.000000,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,1,0,0,0,0,0,1,0,0,1
2,4.000000,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,1,0,0,0,0,0,1,0,0,1
3,5.000000,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,1,0,0,0,0,0,1,0,1,1
4,5.000000,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,0.0,0.0,219.000000,36.000000,1.300000,139.000000,3.700000,12.500000,...,1,0,0,0,0,0,1,0,0,1
395,51.492308,70.000000,0.0,2.0,220.000000,68.000000,2.800000,137.528754,4.627244,8.700000,...,1,0,0,1,1,0,1,0,1,1
396,51.492308,70.000000,3.0,0.0,110.000000,115.000000,6.000000,134.000000,2.700000,9.100000,...,1,0,0,1,1,0,0,0,0,1
397,51.492308,90.000000,0.0,0.0,207.000000,80.000000,6.800000,142.000000,5.500000,8.500000,...,1,0,0,1,1,0,1,0,1,1


In [6]:
dataset["classification_yes"].value_counts()
indep=dataset[["age","bp","al","su","bgr","bu","sc","sod","pot","hrmo","pcv","wc","rc","sg_b","sg_c","sg_d","sg_e","rbc_normal","pc_normal","pcc_present","ba_present","htn_yes","dm_yes","cad_yes","appet_yes","pe_yes","ane_yes"]]
dep=dataset[["classification_yes"]]

In [7]:
from sklearn.model_selection import train_test_split
X_train,x_test,y_train,y_test=train_test_split(indep,dep,test_size=0.30,random_state=0)

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
param_grid={"criterion":["gini",'entropy'],
            "max_features":['auto','sqrt','log2'],"n_estimators":[10,100]}
grid=GridSearchCV(RandomForestClassifier(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring="f1")
grid.fit(X_train,y_train)

C:\Users\LENOVO\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    7.2s finished
C:\Users\LENOVO\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:715: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
           

In [9]:
re=grid.cv_results_
grid_predict=grid.predict(x_test)

In [10]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_predict)

In [12]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_predict)
print(clf_report)


              precision    recall  f1-score   support

           0       0.98      0.98      0.98        45
           1       0.99      0.99      0.99        75

    accuracy                           0.98       120
   macro avg       0.98      0.98      0.98       120
weighted avg       0.98      0.98      0.98       120



In [13]:
print(cm)

[[44  1]
 [ 1 74]]


In [14]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.083355,0.014754,0.010417,0.007366,gini,auto,10,"{'criterion': 'gini', 'max_features': 'auto', ...",0.956522,0.935780,0.991453,0.961252,0.022973,12
1,0.265627,0.000043,0.026007,0.007341,gini,auto,100,"{'criterion': 'gini', 'max_features': 'auto', ...",0.982759,0.964286,0.983051,0.976698,0.008778,4
2,0.041620,0.007371,0.000000,0.000000,gini,sqrt,10,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.983051,0.935780,0.983051,0.967294,0.022284,11
3,0.239574,0.007354,0.026040,0.007366,gini,sqrt,100,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.983051,0.964286,0.983051,0.976796,0.008846,3
4,0.062499,0.022097,0.005208,0.007365,gini,log2,10,"{'criterion': 'gini', 'max_features': 'log2', ...",0.991453,0.935780,0.983051,0.970095,0.024505,10
5,0.260427,0.007349,0.031262,0.012739,gini,log2,100,"{'criterion': 'gini', 'max_features': 'log2', ...",0.982759,0.964286,0.983051,0.976698,0.008778,4
6,0.046883,0.012746,0.010428,0.007373,entropy,auto,10,"{'criterion': 'entropy', 'max_features': 'auto...",0.982759,0.945455,0.983051,0.970421,0.017655,9
7,0.254376,0.007377,0.020839,0.007359,entropy,auto,100,"{'criterion': 'entropy', 'max_features': 'auto...",0.974359,0.954955,0.983051,0.970788,0.011745,8
8,0.023540,0.010142,0.010423,0.007370,entropy,sqrt,10,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.982759,0.954955,0.983051,0.973588,0.013176,7
9,0.280958,0.006477,0.022670,0.006473,entropy,sqrt,100,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.991453,0.964286,0.983051,0.979597,0.011357,2


In [15]:
Future_predict=grid.predict([[2,76,3,0,148.1,57.8,3.08,137.53,4.63,12.52,38.87,8403.3,4.71,0,0,0,0,1,0,0,0,0,0,0,1,1,0]])
Future_predict

array([1], dtype=uint8)